In [1]:
import pandas as pd

In [2]:
OI_train_file = 'data/Open_Images/oidv6-train-images-with-labels-with-rotation.parquet'
train_df = pd.read_parquet(OI_train_file)

print("OI_train_file columns:", train_df.columns)
print("OI_train_file shape:", train_df.shape)
train_df.head()

OI_train_file columns: Index(['ImageID', 'Subset', 'OriginalURL', 'OriginalLandingURL', 'License',
       'AuthorProfileURL', 'Author', 'Title', 'OriginalSize', 'OriginalMD5',
       'Thumbnail300KURL', 'Rotation'],
      dtype='object')
OI_train_file shape: (7337077, 12)


,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,d05c3e451f79174d,train,https://c6.staticflickr.com/8/7375/13791738583...,https://www.flickr.com/photos/jeepersmedia/137...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/jeepersmedia/,Mike Mozart,Pez Kids Breakfast Cereal Prizes,972558,KoVMrVWKftzadg1EGr6nCg==,https://c1.staticflickr.com/8/7375/13791738583...,0.0
2,5bde9ad21e82deed,train,https://c6.staticflickr.com/1/223/495658260_89...,https://www.flickr.com/photos/8106459@N07/4956...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/8106459@N07/,David~O,Isabella Lake,234199,dsfj7rNAbGq67OzPRAGdsQ==,https://c2.staticflickr.com/1/223/495658260_85...,0.0
3,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
4,3231e75bd7120b7a,train,https://c3.staticflickr.com/4/3654/3619297168_...,https://www.flickr.com/photos/cckaiser/3619297168,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/cckaiser/,Colin,DSC_0096,2570210,195AcCHMqin3MYTv+pTDuA==,https://c4.staticflickr.com/4/3654/3619297168_...,0.0


In [3]:
OI_label_file = 'data/Open_Images/oidv7-train-annotations-human-imagelabels.parquet'
label_df = pd.read_parquet(OI_label_file)

print("OI_label_file columns:", label_df.columns)
print("OI_label_file shape:", label_df.shape)
label_df.head()

OI_label_file columns: Index(['ImageID', 'Source', 'LabelName', 'Confidence'], dtype='object')
OI_label_file shape: (58783034, 4)


,ImageID,Source,LabelName,Confidence
0,000002b66c9c498e,verification,/m/010l12,0.0
1,000002b66c9c498e,verification,/m/0118n_9r,0.0
2,000002b66c9c498e,verification,/m/011_f4,0.0
3,000002b66c9c498e,verification,/m/011l78,0.0
4,000002b66c9c498e,verification,/m/011s0,0.0


In [18]:
mini_imagenet_class_df = pd.read_csv('data/Open_Images/mini_imagenet_oi_classes.csv')
non_mini_imagenet_class_df = pd.read_csv('data/Open_Images/non_mini_imagenet_oi_classes.csv')

print(f"mini_imagenet_class_df shape: {mini_imagenet_class_df.shape}")
print(f"non_mini_imagenet_class_df shape: {non_mini_imagenet_class_df.shape}")

mini_imagenet_class_df.head()

mini_imagenet_class_df shape: (726, 2)
non_mini_imagenet_class_df shape: (20163, 2)


,LabelName,DisplayName
0,/m/08y7p8,Abdomen
1,/m/0hgntgw,Active tank
2,/m/0h8p55r,Address sign
3,/m/011l1,Afghan hound
4,/m/0brytpf,Agent orange


In [19]:
train_df['Rotation'].value_counts()

Rotation
0.0      6115548
270.0      95526
90.0       45440
180.0      10241
Name: count, dtype: int64

In [20]:
train_df = train_df[train_df['Rotation'] == 0.0]
print(train_df.shape)

(6115548, 12)


In [21]:
size_df = train_df['OriginalSize'].value_counts().sort_index(ascending=False, inplace=False)

size_df


OriginalSize
15727733    1
15727673    1
15727633    1
15727581    1
15727472    1
           ..
3446        1
2822        1
2555        1
2498        1
1804        1
Name: count, Length: 3434017, dtype: int64

In [22]:
# Lower bound: 10 KB (keeps reasonable photos)
# Upper bound: 5 MB (protects against weird corrupted multimegabyte files)
clean_train_df = train_df[(train_df['OriginalSize'] > 10_000) & (train_df['OriginalSize'] < 5_000_000)]

print(clean_train_df.shape)

(5328886, 12)


In [23]:
label_df['Source'].value_counts()

Source
verification                48965561
crowdsource-verification     9817473
Name: count, dtype: int64

In [24]:
label_df['Confidence'].value_counts()

Confidence
0.0    37638859
1.0    21144175
Name: count, dtype: int64

In [25]:
label_df = label_df[label_df['Confidence'] == 1.0]

label_df.shape

(21144175, 4)

In [26]:
inhouse_label_df = label_df[label_df['Source'] == 'verification']

inhouse_label_df.shape

(14703564, 4)

In [27]:
outsourced_label_df = label_df[label_df['Source'] == 'crowdsource-verification']

outsourced_label_df.shape

(6440611, 4)

In [28]:
imagenet_label_df = inhouse_label_df.merge(mini_imagenet_class_df, on='LabelName', how='inner')

print(imagenet_label_df.shape)
print("imagenet_label_df ImageID unique:", imagenet_label_df["ImageID"].nunique())
imagenet_label_df.head()

(1819017, 5)
imagenet_label_df ImageID unique: 1532345


,ImageID,Source,LabelName,Confidence,DisplayName
0,000002b66c9c498e,verification,/m/012mj,1.0,Alcoholic beverage
1,000002b66c9c498e,verification,/m/02pkb8,1.0,Lady
2,000002b66c9c498e,verification,/m/030zfn,1.0,Parallel
3,000002b66c9c498e,verification,/m/039xj_,1.0,Human ear
4,000002b66c9c498e,verification,/m/03q69,1.0,Human hair


In [29]:
non_imagenet_label_df = inhouse_label_df.merge(non_mini_imagenet_class_df, on='LabelName', how='inner')

print(non_imagenet_label_df.shape)
print("non_imagenet_label_df ImageID unique:", non_imagenet_label_df["ImageID"].nunique())
non_imagenet_label_df.head()

(12712821, 5)
non_imagenet_label_df ImageID unique: 5018464


,ImageID,Source,LabelName,Confidence,DisplayName
0,000002b66c9c498e,verification,/m/011xdj,1.0,Meringue
1,000002b66c9c498e,verification,/m/012yh1,1.0,Style
2,000002b66c9c498e,verification,/m/014sv8,1.0,Human eye
3,000002b66c9c498e,verification,/m/015c4z,1.0,Sitting
4,000002b66c9c498e,verification,/m/016c68,1.0,Adult


In [33]:
mini_ids = set(imagenet_label_df['ImageID'])
non_mini_ids = set(non_imagenet_label_df['ImageID'])

print("imagenet_label_df ImageID unique:", len(mini_ids))
print("non_imagenet_label_df ImageID unique:", len(non_mini_ids))

print("Images with only mini-imagenet labels:", len(mini_ids - non_mini_ids))
print("Images with only non-mini-imagenet labels:", len(non_mini_ids - mini_ids))
print("Images with both mini-imagenet and non-mini-imagenet labels:", len(mini_ids & non_mini_ids))

imagenet_label_df ImageID unique: 1532345
non_imagenet_label_df ImageID unique: 5018464
Images with only mini-imagenet labels: 327845
Images with only non-mini-imagenet labels: 3813964
Images with both mini-imagenet and non-mini-imagenet labels: 1204500


In [48]:
mini_only_ids = mini_ids - non_mini_ids
non_only_ids = non_mini_ids - mini_ids
shared_ids = mini_ids & non_mini_ids

mini_only_df = imagenet_label_df[imagenet_label_df['ImageID'].isin(mini_only_ids)].copy()
non_only_df = non_imagenet_label_df[non_imagenet_label_df['ImageID'].isin(non_only_ids)].copy()

mixed_mini_df = imagenet_label_df[imagenet_label_df['ImageID'].isin(shared_ids)].copy()
mixed_mini_df['LabelType'] = 'mini-imagenet'

mixed_non_df = non_imagenet_label_df[non_imagenet_label_df['ImageID'].isin(shared_ids)].copy()
mixed_non_df['LabelType'] = 'non-mini-imagenet'

mixed_label_df = pd.concat([mixed_mini_df, mixed_non_df], ignore_index=True)

print(f"mini_only_df shape: {mini_only_df.shape}")
print(f"non_only_df shape: {non_only_df.shape}")
print(f"mixed_label_df shape: {mixed_label_df.shape}")

mixed_label_df.head()

mini_only_df shape: (345773, 5)
non_only_df shape: (8412184, 5)
mixed_label_df shape: (5773881, 6)


,ImageID,Source,LabelName,Confidence,DisplayName,LabelType
0,000002b66c9c498e,verification,/m/012mj,1.0,Alcoholic beverage,mini-imagenet
1,000002b66c9c498e,verification,/m/02pkb8,1.0,Lady,mini-imagenet
2,000002b66c9c498e,verification,/m/030zfn,1.0,Parallel,mini-imagenet
3,000002b66c9c498e,verification,/m/039xj_,1.0,Human ear,mini-imagenet
4,000002b66c9c498e,verification,/m/03q69,1.0,Human hair,mini-imagenet


In [49]:
mini_only_n = mini_only_df['ImageID'].nunique()
non_only_n = non_only_df['ImageID'].nunique()
mixed_n = mixed_label_df['ImageID'].nunique()

print(mini_only_n)
print(non_only_n)
print(mixed_n)

assert len(mini_ids - non_mini_ids) == mini_only_n
assert len(non_mini_ids - mini_ids) == non_only_n
assert len(mini_ids & non_mini_ids) == mixed_n


327845
3813964
1204500


In [ ]:
# Group by ImageID and aggregate DisplayName into a list
grouped_mini_only_df = mini_only_df.groupby('ImageID')['DisplayName'].apply(list).reset_index(name='Labels')
grouped_non_only_df = non_only_df.groupby('ImageID')['DisplayName'].apply(list).reset_index(name='Labels')
grouped_mixed_df = mixed_label_df.groupby('ImageID')['DisplayName'].apply(list).reset_index(name='Labels')

print(grouped_mini_only_df.shape)
print(grouped_non_only_df.shape)
print(grouped_mixed_df.shape)

assert grouped_mini_only_df.shape[0] == mini_only_n
assert grouped_non_only_df.shape[0] == non_only_n
assert grouped_mixed_df.shape[0] == mixed_n

grouped_mixed_df.head()


(327845, 2)
(3813964, 2)
(1204500, 2)


,ImageID,Labels
0,000002b66c9c498e,"[Alcoholic beverage, Lady, Parallel, Human ear..."
1,000002b97e5471a0,"[Parallel, Street light, Tan, Orange (Color), ..."
2,000002c707c9895e,"[Organ (Biology), Headgear, Beard, Craft, Lady..."
3,0000048549557964,"[Street, Parallel, Green, Street sign, Rear-vi..."
4,000004f4400f6ec5,"[Organ (Biology), Beauty, Hearing, Standing, P..."


In [54]:
final_mini_only_df = clean_train_df[["ImageID", "Subset"]].merge(grouped_mini_only_df, on="ImageID", how="inner")
final_non_only_df = clean_train_df[["ImageID", "Subset"]].merge(grouped_non_only_df, on="ImageID", how="inner")
final_mixed_df = clean_train_df[["ImageID", "Subset"]].merge(grouped_mixed_df, on="ImageID", how="inner")

print(final_mini_only_df.shape)
print(final_non_only_df.shape)
print(final_mixed_df.shape)

final_mixed_df.head()

(234598, 3)
(2825344, 3)
(891316, 3)


,ImageID,Subset,Labels
0,4fa8054781a4c382,train,"[Organ (Biology), Standing, Pray, Adult, Chape..."
1,d05c3e451f79174d,train,"[Beard, Box, Human hair, Man, Advertising, Toy..."
2,b37f763ae67d0888,train,"[Standing, Green, Man, Social group, Grassland..."
3,3231e75bd7120b7a,train,"[Beauty, Parallel, Style, Black, Black-and-whi..."
4,68e7e9623b91ccb3,train,"[Roof, Tree, Water resources, Balcony, Infrast..."


In [55]:
final_mini_only_df = final_mini_only_df.sample(frac=1, random_state=42).reset_index(drop=True)
final_non_only_df = final_non_only_df.sample(frac=1, random_state=42).reset_index(drop=True)
final_mixed_df = final_mixed_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Shuffled final_mini_only_df shape: {final_mini_only_df.shape}")
print(f"Shuffled final_non_only_df shape: {final_non_only_df.shape}")
print(f"Shuffled final_mixed_df shape: {final_mixed_df.shape}")

final_mixed_df.head()

Shuffled final_mini_only_df shape: (234598, 3)
Shuffled final_non_only_df shape: (2825344, 3)
Shuffled final_mixed_df shape: (891316, 3)


,ImageID,Subset,Labels
0,2d465420f07f63d3,train,"[Long underwear, Baseball (sport)]"
1,26df37f08c50d159,train,"[Man, Tree, Person, Sand wedge, Golfer, Golf c..."
2,aa03526cb28bd5e8,train,"[Car, Vehicle]"
3,fc8a6224ceb33662,train,"[Outerwear, Person, Plant]"
4,048ddaf4cd5367ea,train,"[Tree, Fisheye lens, Human head, Mammal, Plant..."


In [56]:
import os
from pathlib import Path
OI_dir = Path("data/Open_Images")

os.makedirs(OI_dir, exist_ok=True)
mini_only_save_file = OI_dir / "mini_only_train_list.parquet"
non_only_save_file = OI_dir / "non_only_train_list.parquet"
mixed_save_file = OI_dir / "mixed_train_list.parquet"

assert not os.path.exists(mini_only_save_file), f"File {mini_only_save_file} already exists"
assert not os.path.exists(non_only_save_file), f"File {non_only_save_file} already exists"
assert not os.path.exists(mixed_save_file), f"File {mixed_save_file} already exists"

final_mini_only_df.to_parquet(mini_only_save_file, index=False)
final_non_only_df.to_parquet(non_only_save_file, index=False)
final_mixed_df.to_parquet(mixed_save_file, index=False)